see README for how I generated the first set of training data (via Prodigy)

this is how I'm generating training data from the RSS database.

In [13]:
from os import environ, makedirs
from os.path import join, exists, dirname
import requests
from urllib.parse import urlencode
import logging
import csv
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
from os import environ
from dotenv import load_dotenv
load_dotenv()

live_engine = create_engine(environ.get("LIVE_DATABASE_URL"))


In [20]:
SEARCH_WARRANT_CSV_FN = 'search_warrants_model_training/search_warrants_for_ner_from_rss.csv'
with open(SEARCH_WARRANT_CSV_FN, 'w') as outcsv:
    writer = None
            
    sw_docs = pd.read_sql("""select case_name, min(document_type) document_type, case_number, court from rss_docket_entries where substring(case_number, 6, 2) = 'sw' group by case_name, case_number, court""", live_engine)
    for i, result in sw_docs.iterrows():
        if not writer:
            writer = csv.DictWriter(outcsv, fieldnames=result.to_dict().keys())
            writer.writeheader()
        writer.writerow(result.to_dict())

    csvs = [
            "search_warrants_model_training/search warrant training data - rss_document_types_all_courts.csv", 
    ]
    for csvfn in csvs:
        rows = pd.read_csv(csvfn)
        rows = rows[rows["is search warrant?"]]
        rows = rows[["case_name", "document_type", "case_number", "court"]]
        for i, result in rows.iterrows():
            if not writer:
                writer = csv.DictWriter(outcsv, fieldnames=result.to_dict().keys())
                writer.writeheader()
            writer.writerow(result.to_dict())



In [21]:
!grep 'USA v. Information' search_warrants_model_training/search_warrants_for_ner_from_rss.csv